# CIS4930 -- Final Project
## Developed by: Chloe Fandino (Team Leader), Ashley James, Madelyne Wirbel, Chloe Nolan, Christopher Enlow

### Imports

In [ ]:
# imports here :)

# TODO: DELETE ---> any imports that don't end up getting used by the end of the project !!!!

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt

### Loading the data

In [ ]:
df = pd.read_csv('OnlineNewsPopularity.csv') # import the data from the csv file --> convert to df

### Exploration of the dataset

In [ ]:
# pd.set_option('display.max_rows', 100) # for purposes of looking at data --> need to see all rows

# basic intitial looks at the dataset
print(df.shape)

print(df.info())

In [ ]:
# Strip whitespace from column names
df.columns = df.columns.str.strip()

In [ ]:
print(df.columns.tolist()) # print out all of the available columns

In [ ]:
df.isnull().sum() # null values? --> NONE :)

In [ ]:
df.duplicated().sum() # duplicate values? --> NONE :)

In [ ]:
# check if any infinities exist in the dataframe
numeric_df = df.select_dtypes(include=[np.number])

has_inf = np.isinf(numeric_df.to_numpy()).any()

print(has_inf) # will need to handle in cleaning
inf_cols = numeric_df.columns[np.isinf(numeric_df.to_numpy()).any(axis=0)].tolist()
print("Columns with inf:", inf_cols)

### Cleaning the data

Drop non-predictive columns

In [ ]:
# exploration of potentially non-predictive features
df['url'].nunique() == len(df) # each example has a different url --> not needed in the dataset
df = df.drop(columns = ['url'])

df = df.drop(columns = ['timedelta']) # number of days since posted until added to the dataset --> no predictive qualities

Identify binary columns and ensure they are ints

In [ ]:
binary_cols = [col for col in df.columns 
               if col.startswith("data_channel_is_") or col.startswith("weekday_is_")]

# ensure binary indicator columns are integers
df.columns = df.columns.str.strip()
print("Binary indicator columns:", binary_cols)

for col in binary_cols:
    df[col] = df[col].astype(int)

df[binary_cols].dtypes.head()

Handle inf/NaN

In [ ]:
# handling inf
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()
print("Dataset shape after removing inf/NaN:", df.shape)

Remove highly correlated features

In [ ]:
# test of feature correlation --> highly correlated features do not need to both be in dataser
df_correlation = df.corr()

sns.heatmap(df_correlation, cmap = 'coolwarm', center = 0)
plt.title("Correlation Heatmap")

In [ ]:
X_corr = df.drop(columns=['shares'])
y_corr = df['shares']

def drop_correlated_by_importance(X, y, threshold=0.7, n_estimators=50, protected_cols=[]):
    model = ExtraTreesRegressor(n_estimators=n_estimators, random_state=42, n_jobs=-1)
    model.fit(X, y)
    importances = pd.Series(model.feature_importances_, index=X.columns)

    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    corr_pairs = upper.stack().reset_index()
    corr_pairs.columns = ['feat1', 'feat2', 'corr']
    corr_pairs = corr_pairs[corr_pairs['corr'] > threshold]

    to_drop = set()
    for _, row in corr_pairs.iterrows():
        if row['feat1'] in protected_cols or row['feat2'] in protected_cols: # we dont want to drop binary indicator cols
            continue
        if importances[row['feat1']] >= importances[row['feat2']]:
            to_drop.add(row['feat2'])
        else:
            to_drop.add(row['feat1'])
    return list(to_drop)

In [ ]:
columns_to_drop = drop_correlated_by_importance(
    X_corr, y_corr, threshold=0.6, n_estimators=50, protected_cols=binary_cols
)

print("Columns to drop due to high correlation:", columns_to_drop)
df = df.drop(columns=columns_to_drop)

### Winsorization

In [ ]:
# Identify numeric non-binary indicator columns
numeric_cols = [c for c in df.columns if c not in binary_cols + ['shares']]

# Winsorize
for col in numeric_cols:
    print(f"Before winsorizing {col}: min={df[col].min()}, max={df[col].max()}")
    df[col] = winsorize(df[col], limits=[0.01, 0.01])
    print(f"After winsorizing {col}: min={df[col].min()}, max={df[col].max()}\n")

In [ ]:
# function with anomaly detection process --> first visualize, then calculate IQR

## Currently not used in the final cleaning process, but kept for potential future use ##

def anomaly_detection(feature):
    # first boxplot to see potential outliers
    sns.boxplot(x = df[feature], color = 'purple')
    plt.title(feature)
    plt.show()

    # second calculate outliers based on IQR
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)

    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    anomalies = df[(df[feature] < lower) | (df[feature] > upper)]
    print('Anomalies: \n', anomalies) # prints a list of potential anomalies
    return

## Training and Testing

### Create Training Split

In [ ]:
# separate features
y = df["shares"]
X = df.drop(columns = ["shares"])

print(y.value_counts())

print("X shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set:", X_train.shape)
print("Test set:", X_test.shape)

In [ ]:
# scale the data
scaler = RobustScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Skewness before log-transform:", pd.Series(y_train).skew())

y_train_knn = y_train
y_test_knn = y_test
y_train = np.log1p(y_train)
y_test = np.log1p(y_test)

# Check skewness after log-transform
print("Skewness after log-transform:", pd.Series(y_train).skew())

sns.histplot(y_train, kde=True)
plt.title("Log-transformed target distribution")
plt.show()

### 1. Linear Regression Model

In [ ]:
# train linear regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)
y_pred = lin_reg.predict(X_test_scaled)

In [ ]:
# evaluate linear regression performance

### 2. Neural Network

In [ ]:
# Convert features + targets to tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test_scaled, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # already log-scaled
y_test_tensor  = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)   # already log-scaled

# DataLoaders
train_loader = DataLoader(
    TensorDataset(X_train_tensor, y_train_tensor),
    batch_size=64,
    shuffle=True
)

test_loader = DataLoader(
    TensorDataset(X_test_tensor, y_test_tensor),
    batch_size=64,
    shuffle=False
)

# model definition
class RegressionNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

model = RegressionNN(input_dim=X_train_tensor.shape[1])

criterion = nn.MSELoss()  # MSE in log-space
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # slower lr for stability
n_epochs = 300

train_losses = []
test_losses = []

for epoch in range(n_epochs):
    model.train()
    running_train_loss = 0.0

    # training step
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item() * X_batch.size(0)

    train_epoch_loss = running_train_loss / len(train_loader.dataset)
    train_losses.append(train_epoch_loss)

    # test step
    model.eval()
    running_test_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            test_loss = criterion(y_pred, y_batch)
            running_test_loss += test_loss.item() * X_batch.size(0)

    test_epoch_loss = running_test_loss / len(test_loader.dataset)
    test_losses.append(test_epoch_loss)

    # print every 20 epochs
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{n_epochs}] "
              f"Train Loss: {train_epoch_loss:.4f} | "
              f"Test Loss: {test_epoch_loss:.4f}")

# plot loss curves
plt.figure(figsize=(8,5))
plt.plot(train_losses, label="Train Loss (log-space)")
plt.plot(test_losses, label="Test Loss (log-space)")
plt.xlabel("Epoch")
plt.ylabel("MSE (log-space)")
plt.title("Training vs Test Loss")
plt.legend()
plt.grid(True)
plt.show()

# evaluation on original scale
model.eval()
with torch.no_grad():
    y_pred_log = model(X_test_tensor).numpy()
    y_pred = np.expm1(y_pred_log).reshape(-1)             # invert log
    y_true = np.expm1(y_test_tensor.numpy()).reshape(-1)   # invert log

print("\nFINAL RESULTS:")
print("Test MSE:", mean_squared_error(y_true, y_pred))
print("Test R2: ", r2_score(y_true, y_pred))
print("Test MAE:", mean_absolute_error(y_true, y_pred))

### 3. KNN

In [ ]:
k_values = range(1, 40)
mse_scores = []

print("Finding optimal K using 'distance' weights...")

for k in k_values:
    # CRITICAL: We use weights='distance' inside the loop too
    knn = KNeighborsRegressor(n_neighbors=k, weights='distance')
    knn.fit(X_train_scaled, y_train_knn.values.ravel())
    
    # Predict on test set
    pred = knn.predict(X_test_scaled)
    
    # Calculate MSE (on raw values now)
    mse = mean_squared_error(y_test.values, pred)
    mse_scores.append(mse)

# Plot the Elbow Curve
plt.figure(figsize=(8,5))
plt.plot(k_values, mse_scores, marker='o', linestyle='dashed')
plt.xlabel("K Value")
plt.ylabel("MSE (Lower is Better)")
plt.title("Finding Optimal K (Distance Weighted)")
plt.grid(True)
plt.show()

# Output the best K automatically
best_k = k_values[np.argmin(mse_scores)]
print(f"Best K found: {best_k}")

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 1. Define the model using the Best K found above
# weights='distance' is REQUIRED to predict high values in your dataset
knn_best = KNeighborsRegressor(n_neighbors=best_k, weights='distance')

# 2. Fit the model
knn_best.fit(X_train_scaled, y_train_knn.values.ravel()) 

# 3. Prediction (Directly in raw scale)
y_pred = knn_best.predict(X_test_scaled)
y_true = y_test_knn.values.ravel()

# 4. Evaluation
print(f"--- FINAL RESULTS (K={best_k}, Distance Weighted) ---")
print("Max Actual Value:   ", y_true.max())
print("Max Predicted Value:", y_pred.max()) # This should now be much closer to the max actual
print("-" * 30)
print("Test MSE:", mean_squared_error(y_true, y_pred))
print("Test R2: ", r2_score(y_true, y_pred))
print("Test MAE:", mean_absolute_error(y_true, y_pred))

In [ ]:
plt.figure(figsize=(10, 6))

# Scatter plot
sns.scatterplot(x=y_true, y=y_pred, alpha=0.6, color='blue', edgecolor='k', label='Predictions')

# Perfect fit line
min_val = min(y_true.min(), y_pred.min())
max_val = max(y_true.max(), y_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', lw=2, label='Perfect Fit')

plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title(f"KNN Regression Results (k={best_k}): Predicted vs. Actual")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Choose which feature to visualize (0 is the first column)
feature_index = 0 
feature_name = "Feature 0" # Rename this to "Square Footage" or "Income" etc. if you know it

# Sort data for clean plotting
sort_idx = X_test_scaled[:, feature_index].argsort()
X_sorted = X_test_scaled[sort_idx]
y_pred_sorted = y_pred[sort_idx]
y_true_sorted = y_true[sort_idx]

plt.figure(figsize=(10, 6))

# 1. Plot the actual real data (Grey dots)
plt.scatter(X_sorted[:, feature_index], y_true_sorted, color='gray', alpha=0.4, label='Actual Data')

# 2. Plot the KNN prediction (Red line)
plt.plot(X_sorted[:, feature_index], y_pred_sorted, color='red', linewidth=2, label=f'KNN Prediction (k={best_k})')

plt.xlabel(f"Normalized {feature_name}")
plt.ylabel("Target Value")
plt.title(f"KNN Model Fit on {feature_name}")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
print("--- SANITY CHECK ---")
print(f"Sample Actual (y_true): {y_true[:5]}")
print(f"Sample Predicted (y_pred): {y_pred[:5]}")
print(f"Max Actual: {y_true.max()}")
print(f"Max Predicted: {y_pred.max()}")